In [11]:
import pandas as pd

#implementing improving search for line production

#reading the data 
df=pd.read_excel('Line Production September 2023.xlsx')
df.head(27)

,Product,L1,L2,L3,L4,L5,L6,L7,deadline,penalty cost
0,P1,32,43,41,67,50,91,26,108,43
1,P2,66,100,61,31,32,24,78,57,6
2,P3,87,83,54,80,16,72,25,118,6
3,P4,60,85,81,16,93,86,31,71,48
4,P5,73,79,54,65,69,44,54,81,44
5,P6,20,47,52,65,66,59,23,57,27
6,P7,71,25,73,53,31,93,54,83,82
7,P8,45,34,67,25,57,35,40,105,36
8,P9,33,83,64,69,46,92,100,96,41
9,P10,82,19,81,27,32,25,84,53,8


In [15]:
# Sort the products by their deadlines (ascending order)
df = df.sort_values(by='deadline')

# Initialize variables to keep track of the schedule and product completion times on each line
lines = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7']
product_schedule = {line: [] for line in lines}
product_completion_time = {line: 0 for line in lines}

# Iterate through the products and assign them to lines
for _, row in df.iterrows():
    product = row['Product']
    line_assigned = None
    min_completion_time = float('inf')

    # Find the line with the earliest completion time for the current product
    for line in lines:
        completion_time = max(product_completion_time[line], row[line])
        if completion_time < min_completion_time:
            min_completion_time = completion_time
            line_assigned = line

    # Assign the product to the chosen line and update the completion time
    product_schedule[line_assigned].append(product)
    product_completion_time[line_assigned] = min_completion_time + row[line_assigned]

# Print the schedule for each line
for line, products in product_schedule.items():
    print(f'{line}: {products}')

# Calculate the total penalty cost
total_penalty_cost = df[df['Product'].isin(product_schedule[line])]['penalty cost'].sum()
print(f'Total Penalty Cost: {total_penalty_cost}')

L1: ['P26', 'P17', 'P18', 'P20']
L2: ['P10', 'P7', 'P27', 'P19', 'P12']
L3: ['P13', 'P14', 'P22', 'P15']
L4: ['P11', 'P9', 'P25']
L5: ['P23', 'P16', 'P21']
L6: ['P2', 'P5', 'P8', 'P1']
L7: ['P6', 'P4', 'P24', 'P3']
Total Penalty Cost: 121
